In [ ]:
# -*- coding: utf-8 -*-
"""
Corrected Federated CNN for PCOS detection (full script)
- Proper FedAvg: each client trains independently from the same global weights.
- Weighted averaging uses the number of samples per client.
- Hyperparameters set as requested.
"""

import os
import glob
import random
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import SGD
import tensorflow as tf

In [ ]:
# -----------------------------
# Hyperparameters (user-specified)
# -----------------------------
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 128
NUM_CLIENTS = 10
LOCAL_EPOCHS = 2
COMMS_ROUNDS = 20
LEARNING_RATE = 0.0001
NUM_CLASSES = 2
RANDOM_STATE = 42


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# -----------------------------
# Dataset paths - keep your folder structure
# dataset/normal/* and dataset/pcos/*
# -----------------------------
NORMAL_PATH = '/content/drive/MyDrive/PCOS MINOR/data/train/notinfected/*.*'
PCOS_PATH = '/content/drive/MyDrive/PCOS MINOR/data/train/infected/*.*'


In [ ]:
# -----------------------------
# Load images and labels
# -----------------------------
print("Loading images...")
controlled_files = glob.glob(NORMAL_PATH)
diseased_files = glob.glob(PCOS_PATH)

data = []
labels = []

for p in controlled_files:
    img = load_img(p, target_size=IMAGE_SIZE)
    data.append(np.array(img))
    labels.append(0)

for p in diseased_files:
    img = load_img(p, target_size=IMAGE_SIZE)
    data.append(np.array(img))
    labels.append(1)

data = np.array(data)
labels = np.array(labels)
print("Total images loaded:", data.shape[0])

# one-hot encode labels
categorical_labels = to_categorical(labels, num_classes=NUM_CLASSES)


Loading images...
Total images loaded: 1924


In [ ]:
# -----------------------------
# Train / Val / Test split
# -----------------------------
X_train, X_temp, y_train, y_temp = train_test_split(
    data, categorical_labels, test_size=0.2, random_state=random.randint(1,100), stratify=categorical_labels
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=random.randint(1,100), stratify=y_temp
)

# Create normalized test copy (used for global eval)
X_test_norm = X_test.astype('float32') / 255.0
y_test_norm = y_test

print("Train size:", X_train.shape[0], "Val size:", X_val.shape[0], "Test size:", X_test.shape[0])


Train size: 1539 Val size: 192 Test size: 193


In [ ]:
# -----------------------------
# Create client shards (even split, shuffled)
# -----------------------------
def create_client_shards(X, y, num_clients=NUM_CLIENTS, seed=None):
    # shuffle then split into num_clients shards as evenly as possible
    if seed is not None:
        np.random.seed(seed)
    idxs = np.arange(X.shape[0])
    np.random.shuffle(idxs)
    X_shuffled = X[idxs]
    y_shuffled = y[idxs]
    # split into nearly equal parts
    X_splits = np.array_split(X_shuffled, num_clients)
    y_splits = np.array_split(y_shuffled, num_clients)
    return X_splits, y_splits

client_data_splits, client_label_splits = create_client_shards(X_train, y_train, NUM_CLIENTS, seed=RANDOM_STATE)

# confirm shard sizes
for i, shard in enumerate(client_data_splits):
    print(f"Client {i+1} samples: {shard.shape[0]}")


Client 1 samples: 154
Client 2 samples: 154
Client 3 samples: 154
Client 4 samples: 154
Client 5 samples: 154
Client 6 samples: 154
Client 7 samples: 154
Client 8 samples: 154
Client 9 samples: 154
Client 10 samples: 153


In [ ]:
def build_deeper_cnn(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3), num_classes=NUM_CLASSES):
    model = Sequential()
    # Conv block 1
    model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=input_shape))
    model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    # Conv block 2
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    # Conv block 3
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    # Conv block 4
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    # Classifier head
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model


In [ ]:
# -----------------------------
# Federated helper functions: scale and aggregate weights (FedAvg)
# -----------------------------
def scale_model_weights(weights, scalar):
    return [w * scalar for w in weights]

def sum_scaled_weights(scaled_weights_list):
    # scaled_weights_list is a list of lists of numpy arrays (weights per client already scaled)
    avg_weights = []
    # zip across layers
    for layer_weights in zip(*scaled_weights_list):
        avg = np.sum(layer_weights, axis=0)
        avg_weights.append(avg)
    return avg_weights

# compute weighting factor for a given client based on its sample count
def weight_scalling_factor(client_shard_sizes):
    total = np.sum(client_shard_sizes)
    return [size / total for size in client_shard_sizes]


In [ ]:
# -----------------------------
# Compile global model
# -----------------------------
global_model = build_deeper_cnn(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3), num_classes=NUM_CLASSES)
opt = SGD(learning_rate=LEARNING_RATE, momentum=0.9)
global_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
global_model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 112, 112, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 28, 28, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 50176)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    12,845,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,018,082 (53.47 MB)

 Trainable params: 14,018,082 (53.47 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# -----------------------------
# ImageDataGenerators (train/val)
# -----------------------------
train_datagen = ImageDataGenerator(
    rescale=1./255.,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255.)


Deep CNN

In [ ]:
# -----------------------------
# Federated training loop (FedAvg)
# -----------------------------
# Precompute client weight fractions
client_sizes = [shard.shape[0] for shard in client_data_splits]
client_weight_fractions = weight_scalling_factor(client_sizes)

print("Starting federated training...")
for round_num in range(COMMS_ROUNDS):
    print("\n--- Communication round:", round_num+1, "/", COMMS_ROUNDS, "---")
    # Get global weights (start point for all clients this round)
    global_weights = global_model.get_weights()

    # Shuffle clients order (optional)
    client_indices = list(range(NUM_CLIENTS))
    random.shuffle(client_indices)

    scaled_local_weights = []   # will hold scaled weights from each client
    # For each client, create a fresh local model, set weights, train, collect weights
    for idx in client_indices:
        client_X = client_data_splits[idx]
        client_y = client_label_splits[idx]

        if client_X.shape[0] == 0:
            # if some shard ended up empty (possible with very small datasets), skip
            print(f"Skipping empty client {idx}")
            # append zeros scaled weights so summation shape preserved? better skip because fraction will be zero
            continue

        # Build fresh local model
        local_model = build_deeper_cnn(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3), num_classes=NUM_CLASSES)
        local_model.compile(optimizer=SGD(learning_rate=LEARNING_RATE, momentum=0.9),
                            loss='categorical_crossentropy', metrics=['accuracy'])
        # set global weights
        local_model.set_weights(global_weights)

        # Prepare data generator for this client
        train_gen = train_datagen.flow(client_X, client_y, batch_size=BATCH_SIZE, shuffle=True)
        val_gen = val_datagen.flow(X_val, y_val, batch_size=BATCH_SIZE, shuffle=False)

        # Train locally
        steps_per_epoch = max(1, int(np.ceil(client_X.shape[0] / BATCH_SIZE)))
        val_steps = max(1, int(np.ceil(X_val.shape[0] / BATCH_SIZE)))
        local_model.fit(
            train_gen,
            steps_per_epoch=steps_per_epoch,
            epochs=LOCAL_EPOCHS,
            validation_data=val_gen,
            validation_steps=val_steps,
            verbose=1
        )

        # get updated local weights and scale them by the client's fraction
        client_fraction = client_weight_fractions[idx]
        local_weights = local_model.get_weights()
        scaled_weights = scale_model_weights(local_weights, client_fraction)
        scaled_local_weights.append(scaled_weights)

        # free local model from memory
        tf.keras.backend.clear_session()

    # Aggregate scaled weights (FedAvg)
    if len(scaled_local_weights) == 0:
        print("No client weights collected this round — skipping aggregation.")
        continue

    new_global_weights = sum_scaled_weights(scaled_local_weights)
    global_model.set_weights(new_global_weights)

    # Evaluate global model on holdout test (normalized)
    eval_loss, eval_acc = global_model.evaluate(X_test_norm, y_test_norm, batch_size=BATCH_SIZE, verbose=0)
    print(f"After round {round_num+1} -> Test Loss: {eval_loss:.4f}, Test Acc: {eval_acc:.4f}")


Starting federated training...

--- Communication round: 1 / 20 ---


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/2
2/2 ━━━━━━━━━━━━━━━━━━━━ 43s 18s/step - accuracy: 0.5303 - loss: 0.6917 - val_accuracy: 0.5990 - val_loss: 0.6917
Epoch 2/2
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.5258 - loss: 0.6919 - val_accuracy: 0.6146 - val_loss: 0.6916
Epoch 1/2
2/2 ━━━━━━━━━━━━━━━━━━━━ 11s 5s/step - accuracy: 0.5849 - loss: 0.6914 - val_accuracy: 0.5990 - val_loss: 0.6917
Epoch 2/2
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 384ms/step - accuracy: 0.4844 - loss: 0.6921 - val_accuracy: 0.6250 - val_loss: 0.6916
Epoch 1/2
2/2 ━━━━━━━━━━━━━━━━━━━━ 11s 5s/step - accuracy: 0.5589 - loss: 0.6926 - val_accuracy: 0.5990 - val_loss: 0.6917
Epoch 2/2
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.5951 - loss: 0.6920 - val_accuracy: 0.6094 - val_loss: 0.6916
Epoch 1/2
2/2 ━━━━━━━━━━━━━━━━━━━━ 11s 5s/step - accuracy: 0.5537 - loss: 0.6917 - val_accuracy: 0.5990 - val_loss: 0.6917
Epoch 2/2
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 341ms/step - accuracy: 0.4887 - loss: 0.6926 - val_accuracy: 0.6250 - val_loss: 0.6916
Epoch 1/2
2/2

In [ ]:
# -----------------------------
# Saving the Model
# -----------------------------
OUTPUT_WEIGHTS_FILE = "/content/drive/MyDrive/PCOS MINOR/Federated Learning using Deep CNN (20 Comm Round, 0.0001 LR, 128 BatchSize).h5"
global_model.save(OUTPUT_WEIGHTS_FILE)
